In [ ]:
import arcpy
import pandas as pd
import os

try:
    
    bd_limites = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\_7_GDBs\20230720_MD_Limites_Territoriales.gdb"
    bd_trabajo = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\_7_GDBs\Diagnostico_Indicador.gdb"

    # ? Parametrización de espacio de trabajo
    arcpy.env.workspace = bd_limites
    # ? Creación de lista de feature class, filtrando por una capa específica
    # ! Variable
    lista_feature = [feature for feature in arcpy.arcpy.ListFeatureClasses() if feature == 'Cuencas_Zonificacion_Hidrografica']

    # ? Se genera un cursor, por medio del módulo da (data access) y se recorre la capa filtrada previamente.
    # ? esto se realiza para almacenar los resultados en una lista y generar un .csv, el cual será la base
    # ? para definir el dominio.

    lcode = []
    ldescription = []
    
    # ! Variable
    with arcpy.da.SearchCursor(lista_feature[0], ['POMCA', 'Codigo']) as cursor:
        for row in cursor:
            lcode.append(row[0])
            ldescription.append(row[0])
        print(" *** Se recorre la capa ***")
        
    # ? Se crea dataframe. La función zip unifica dos listas, la función list, las convierte en una sola lista, la cual es el 
    # ? insumo para la definición del dataframe.
    df_dominio = pd.DataFrame(list(zip(lcode, ldescription)), columns = ['code','description'])
    
      # ! Variable
    ruta_generacion_csv_dominios = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\_11_Dominios"
    nombre_dominio = 'domNombrePOMCA'
    ruta_generacion = str(os.path.join(ruta_generacion_csv_dominios,nombre_dominio)) +'.csv'

    # ? Generación de .csv con dominio a partir de dataframe
    df_dominio.to_csv(ruta_generacion)

    # ? Parametrización de espacio de trabajo
    arcpy.env.workspace = bd_trabajo
    
    # ! Variable
    decripcion_dominio = 'Nombre POMCA'
    # ? Dentro del módulo -management- se usa la función Tabla a Dominio
    arcpy.management.TableToDomain(ruta_generacion, 'code', 'description', arcpy.env.workspace, nombre_dominio, decripcion_dominio, update_option = 'REPLACE')
    
    # ! Variable
    campo_asignacion = 'Nombre_Cuenca_Pomca'

    # ? Asignación del dominio al campo específico de la capa.
    # ! Variable
    for dataset in arcpy.ListDatasets(wild_card='_*'):
        for feature in arcpy.ListFeatureClasses(feature_dataset=dataset):
            for campo in arcpy.ListFields(dataset = feature):
                if campo.name == campo_asignacion:    
                    arcpy.management.AssignDomainToField(feature, campo.name, nombre_dominio)
                    print(f"Se asigna el dominio {nombre_dominio} de la capa {feature}")
                else:
                    print(f"La capa {feature} no tiene el campo requerido")
except EOFError as e:
    print(e)

In [ ]:
class dominios_bd:        
    def __init__(self, 
                 bd_limites, 
                 bd_trabajo, 
                 nombre_feature, 
                 campo_dominio_1, 
                 campo_dominio_2, 
                 ruta_generacion_csv_dominios, 
                 nombre_dominio, 
                 decripcion_dominio,
                 campo_asignacion,
                 wildcard):
                
            self.bd_limites = bd_limites
            self.bd_trabajo = bd_trabajo
            self.nombre_feature = nombre_feature
            self.campo_dominio_1 = campo_dominio_1
            self.campo_dominio_2 = campo_dominio_2
            self.ruta_generacion_csv_dominios = ruta_generacion_csv_dominios
            self.nombre_dominio = nombre_dominio
            self.decripcion_dominio = decripcion_dominio
            self.campo_asignacion = campo_asignacion
            self.wildcard = wildcard
    def generacion_dominio(self):
        arcpy.env.overwriteOutput = True
        
        arcpy.env.workspace = self.bd_limites        
        # ? Se asigna en una valiable la capa que permitirá construir el dominio
        lista_feature = [feature for feature in arcpy.arcpy.ListFeatureClasses() if feature == self.nombre_feature]
        
        lcode = []
        ldescription = []        
        # ? Se seleccionan los campos de la capa que se quieren hagan parte del domino como codigo y descriptor
        with arcpy.da.SearchCursor(lista_feature[0], [self.campo_dominio_1, self.campo_dominio_2]) as cursor:
            for row in cursor:
                lcode.append(row[0])
                ldescription.append(row[0])
        print(" *** Se recorre la capa ***")
        
        # ? Se define el dataframe
        df_dominio = pd.DataFrame(list(zip(lcode, ldescription)), columns = ['code','description'])
        # ? Se parametriza la ruta de salida del archivo .csv
        ruta_generacion = str(os.path.join(self.ruta_generacion_csv_dominios, self.nombre_dominio)) +'.csv'
        # ? Exportación de .csv
        df_dominio.to_csv(ruta_generacion)        
        # ? Parametrización de nuevo espacio de trabajo, donde se asignará el dominio (base de datos oficial de trabajo)
        arcpy.env.workspace = self.bd_trabajo
        # ? Asignación de tabla a dominio
        arcpy.management.TableToDomain(ruta_generacion, 'code', 'description', arcpy.env.workspace, self.nombre_dominio, self.decripcion_dominio, update_option = 'REPLACE')
        

        # ? Bucle de actualización masiva de campo con dominios (cuando aplica y cuando no)
        for dataset in arcpy.ListDatasets(wild_card=self.wildcard):
            for feature in arcpy.ListFeatureClasses(feature_dataset=dataset):
                for campo in arcpy.ListFields(dataset=feature):
                    if campo.name == self.campo_asignacion:    
                        arcpy.management.AssignDomainToField(feature, campo.name, self.nombre_dominio)
                        print(f"Se asigna el dominio {self.nombre_dominio} de la capa {feature}")
                    else:
                        print(f"La capa {feature} no tiene el campo requerido")      
    